# This aim of this notebook is to reproduce the results on the challenger dataset 

Author: Maiko Müller (maiko.mueller@web.de) as project of the SMPE course at MoSiG in INP Grenoble

There have already been two reproductions using
 - Python (https://app-learninglab.inria.fr/gitlab/moocrr-session2/moocrr-reproducibility-study/blob/master/src/Python3/challenger.ipynb)
 - R (https://app-learninglab.inria.fr/gitlab/moocrr-session2/moocrr-reproducibility-study/blob/master/src/R/challenger_R_Rmd_Windows_32bits_rmarkdown_1_10.pdf)
This should be done using using the least amount of common software as to avoid having in common code that might have bugs. Therefore I chose the following software stack:
- OS: Linux. The R study was done with Windows, the Python study in Linux. I don't have access to a Mac.
- Programming Language: Since R and Python are already taken, I will try Rust.
- Environment: Jupyter Notebook. Both other studies use ORG-Mode already. Also, Rust has support for ORG-Mode, through a package called Rustic, but (as far as I understand) it does not support shared data between different code snippets, which makes it entirely unusable for this study.

# Steps to use this notebook
- Install `cargo` and `rustc` using `rustup`. More detailed instructions are at https://www.rust-lang.org/tools/install
- Install the Jupyter Kernel for Rust (https://github.com/google/evcxr/blob/master/evcxr_jupyter/README.md)
  - `cargo install evcxr_jupyter`
  - `evcxr_jupyter --install`

evxcr_jupyeter version 0.4.7 was used for this notebook

# If you are just trying to view this document, use the .html version. The notebook itself doesn't render will on Gitlab and the PDF file is missing graphics. 

# Environment

In [2]:
use std::process::Command;
String::from_utf8(Command::new("uname").arg("-a").output().unwrap().stdout).unwrap()

"Linux MAIKO 5.4.10-arch1-1 #1 SMP PREEMPT Thu, 09 Jan 2020 10:14:29 +0000 x86_64 GNU/Linux\n"

In [3]:
String::from_utf8(Command::new("cargo").arg("--version").output().unwrap().stdout).unwrap()

"cargo 1.39.0 (1c6ec66d5 2019-09-30)\n"

In [4]:
String::from_utf8(Command::new("rustc").arg("--version").output().unwrap().stdout).unwrap()

"rustc 1.39.0 (4560ea788 2019-11-04)\n"

For all used packages, see `Cargo.lock` file in the same folder

# Get the data

In [5]:
// Downloading the data doesn't work because of lifetime issues with the Jupyter repl, but would be ok in regular rust 

// :dep reqwest = "0.9.5"
/*
extern crate reqwest;
let data = reqwest::get("https://app-learninglab.inria.fr/gitlab/moocrr-session2/moocrr-reproducibility-study/raw/master/data/shuttle.csv").unwrap()
    .text().unwrap();
*/

In [6]:
// So since the data is rather small, copy-pasting it is actually possible.
// Loading it from disk doesn't work either with Jupyter

let data = "
Date,Count,Temperature,Pressure,Malfunction
4/12/81,6,66,50,0
11/12/81,6,70,50,1
3/22/82,6,69,50,0
11/11/82,6,68,50,0
4/04/83,6,67,50,0
6/18/82,6,72,50,0
8/30/83,6,73,100,0
11/28/83,6,70,100,0
2/03/84,6,57,200,1
4/06/84,6,63,200,1
8/30/84,6,70,200,1
10/05/84,6,78,200,0
11/08/84,6,67,200,0
1/24/85,6,53,200,2
4/12/85,6,67,200,0
4/29/85,6,75,200,0
6/17/85,6,70,200,0
7/2903/85,6,81,200,0
8/27/85,6,76,200,0
10/03/85,6,79,200,0
10/30/85,6,75,200,2
11/26/85,6,76,200,0
1/12/86,6,58,200,1
";

# Parse the data
The idiomatic way to parse data in Rust is to use to define a struct with the values. The libraries `serde` and `serde_derive` are used deserialize the data into a Rust datatype.

In [7]:
:dep serde_derive = "1.0.103"
:dep serde = "1.0.103"
#[macro_use]
extern crate serde_derive;
#[derive(Debug, Deserialize, Eq, PartialEq, Clone )]
pub struct Row {
     #[serde(rename = "Date")]
     date : String,
     #[serde(rename = "Count")]
     count: u32,
     #[serde(rename = "Temperature")]
     temperature: u32,
     #[serde(rename = "Pressure")]
     pressure: u32,
     #[serde(rename = "Malfunction")]
     malfunction: u32,
 }

In [8]:
// But first we need to load the values as CSV
:dep csv = "1.1.1"
extern crate csv;
use csv::Reader;

In [9]:
// Code adapted from the documentation of the csv crate. It will parse in the data as a Vector of rows.
let mut rdr = Reader::from_reader(data.as_bytes());
let iter = rdr.deserialize();
let result : Vec<Row> = iter.map(|e | e.unwrap()).collect();
println!("{:?}", result[0]); // that's how the data looks now

Row { date: "4/12/81", count: 6, temperature: 66, pressure: 50, malfunction: 0 }


# Let's show what we have
For data visualization, we use plotters, which has support for Jupyter notebook with the feature evcxr

In [10]:
:dep plotters = { version = "0.2.12", default_features = true, features = ["evcxr"] }

In [11]:
extern crate plotters;
use plotters::prelude::*;
use plotters::coord::Shift;
use plotters::prelude::LineSeries;
use std::error::Error;
    
    
pub fn plot(     
    data: &Vec<Row>,
    curve : &(Vec<f64>, Vec<f64>),
    drawing_area: &DrawingArea<SVGBackend, Shift>) 
    -> Result<(), Box<dyn std::error::Error>> 
{
    // the base chart
    let mut chart = ChartBuilder::on(&drawing_area)
        .x_label_area_size(30)
        .y_label_area_size(30)
         .build_ranged(0.3f64..1f64, 0f64..1f64).unwrap();
    // TODO we somehow need to get axis label here, but I don't know how

    // backgrund mesh
    chart.configure_mesh().draw()?;

    // Draw the points
    chart.draw_series(
        data.iter().map(|r| Circle::new((r.temperature as f64 / 100.0, r.malfunction as f64 / r.count as f64),
            5, &RED)),
        )?;
    
    // Ploth the curve
    chart.draw_series( LineSeries::new(
        curve.0.iter().zip(curve.1.iter()).map(|(x,y)| (*x, *y)), &RED
    ))?; 
    
    Ok(()) // Everything went well
} 
    

This following plot has quite a few problems:
 - No axis labels
 - The x-axis (temperature) goes from 0.3 to 0.9 instead as 30 to 90 degrees. This is because (for some reason) the regression library only handles x-values from 0 to 1 correctly. 

In [12]:
evcxr_figure((1000, 300), |root| {
    plot(&result, &(Vec::new(), Vec::new()), &root)
})

The type of the variable rdr was redefined, so was lost.


0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0

# Let's try the logistic regression. We use the library rusty-machine for that
The Algorithms in rusty-machine is 100% written in Rust, so there is no danger of re-using the same C libraries that Python or R might be using for regression.

In [13]:
:dep rusty-machine = "0.5.4"
extern crate rusty_machine;
use rusty_machine::learning::logistic_reg::*;
//use rusty_machine::linalg::Matrix;
//use rusty_machine::linalg::Vector;
use rusty_machine::learning::SupModel;
use rusty_machine::learning::glm::*;
use rusty_machine::learning::optim::grad_desc::{GradientDesc};

:dep rulinalg = "0.3.7"
extern crate rulinalg;
use rulinalg::vector::Vector;
use rulinalg::matrix::{Matrix, BaseMatrix};


// As in the other studies, we try to show that the failures are directly related to the temperature
let temperatures : Vec<_> = result.iter().map(|x| x.temperature as f64 / 100.0).collect();
let frequencies : Vec<_> = result.iter().map(|x| x.malfunction as f64 / x.count as f64).collect();

// We need to convert this into the correct data-types for rusty_machine
let inputs = Matrix::new(temperatures.len(),1, temperatures);
let targets = Vector::new(frequencies);

// The regressor
let mut log_mod = LogisticRegressor::default();

// Train the model
log_mod.train(&inputs, &targets).unwrap();


  

Let's see what do regressor does. We generate x values, predict them and plot that

In [14]:
let xvals : Vec<f64> = (30..90).into_iter().map(|x| x  as f64 / 100.0).collect();

// We predict every temperature (30.90) degrees...
let values : Vec<f64>= log_mod.predict(&Matrix::new(xvals.len(), 1,xvals.clone()))
    .unwrap().iter().cloned().collect();
// ... and draw them
evcxr_figure((1000, 300), |root| {
    plot(&result, &(xvals.clone(), values.clone()), &root)
}) 

0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="30,240 43,240 57,241 71,241 85,241 99,241 113,242 127,242 140,242 154,243 168,243 182,243 196,243 210,244 224,244 237,244 251,245 265,245 279,245 293,245 307,246 321,246 334,246 348,246 362,247 376,247 390,247 404,247 418,247 431,248 445,248 459,248 473,248 487,249 501,249 515,249 528,249 542,249 556,250 570,250 584,250 598,250 612,250 625,251 639,251 653,251 667,251 681,251 695,252 709,252 722,252 736,252 750,252 764,253 778,253 792,253 806,253 819,253 833,253 847,254 "/>

The logistic regression is really not logistic at all :(
There is something wrong in the method here
The documentation of Rusty_machine says that a General Linear Model with a Bernoulli distribution is a logistic model, so let's try that.

In [15]:
let mut log_mod = GenLinearModel::new(Bernoulli);

// Train the model
log_mod.train(&inputs, &targets).unwrap();

let values : Vec<f64>= log_mod.predict(&Matrix::new(xvals.len(), 1,xvals.clone()))
    .unwrap().iter().cloned().collect();

In [16]:
 evcxr_figure((1000, 300), |root| {
    plot(&result, &(xvals.clone(), values.clone()), &root)
}) 

0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="30,84 43,86 57,88 71,90 85,92 99,94 113,96 127,98 140,100 154,102 168,104 182,106 196,109 210,111 224,113 237,115 251,117 265,119 279,122 293,124 307,126 321,128 334,130 348,133 362,135 376,137 390,139 404,141 418,143 431,146 445,148 459,150 473,152 487,154 501,156 515,159 528,161 542,163 556,165 570,167 584,169 598,171 612,173 625,175 639,177 653,179 667,181 681,183 695,185 709,187 722,189 736,190 750,192 764,194 778,196 792,197 806,199 819,201 833,202 847,204 "/>

That's even worse :(

So what's wrong? I then tried the example that sets from the rusty_machine crate and found out that they actually worked, so the library is not totally broken.
I then incrementally changed the example data to resemble my data more to find out where it breaks.
There seem to be two problems in the library:
 - the x-values somehow have to be between 0 and 1. I have no clue why this is the case but experiments with the original x-values (temperatures) have not worked at all). This is already reflection in the above diagrams, where all temperatures are divided by 100
 - Also, the default number of iterations for Gradient Descent in Rusy_Machine is set to only 100 [0]. And who changes defaults?  Usually, the defaults values of chosen to work well for all cases, but not so for my data. 

With changing default and scaling the x-Axis things work better
 
 [0] https://athemathmo.github.io/rusty-machine/doc/src/rusty_machine/src/learning/optim/grad_desc.rs.html#39

In [17]:
let gd = GradientDesc::new(0.3, 500000);
let mut log_mod= LogisticRegressor::new(gd);

log_mod.train(&inputs, &targets).unwrap();

let values : Vec<f64>= log_mod.predict(&Matrix::new(xvals.len(), 1,xvals.clone()))
    .unwrap().iter().cloned().collect();

In [18]:
evcxr_figure((1000, 300), |root| {
    plot(&result, &(xvals.clone(), values.clone()), &root)
}) 

0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="30,43 43,48 57,53 71,58 85,63 99,69 113,75 127,82 140,89 154,96 168,103 182,110 196,118 210,126 224,134 237,141 251,149 265,157 279,164 293,172 307,179 321,185 334,192 348,198 362,204 376,209 390,215 404,219 418,224 431,228 445,232 459,235 473,239 487,242 501,244 515,247 528,249 542,251 556,253 570,254 584,256 598,257 612,258 625,259 639,260 653,261 667,262 681,263 695,263 709,264 722,264 736,265 750,265 764,266 778,266 792,266 806,266 819,267 833,267 847,267 "/>

In [19]:
log_mod

LogisticRegressor { base: BaseLogisticRegressor { parameters: Some(Vector { size: 2, data: [5.084925304946187, -11.560036448787319] }) }, alg: GradientDesc { alpha: 0.3, iters: 500000 } }

This looks a  close to the actual curve from the other reproductions. The measured $\alpha$ and $\beta$ values are 5.0849 and -11.56. Those are exactly the same values as in the study (the second value is 100 times higher because of the scaling in the x-axis, but otherwise it's the same


So, far the other values... I am still missing the standard error G^2 and the degrees of freedom. Both the R and Python implementations get those for free from the statistics library they are using. This is not the case in Rust. To the best of my knowledge there doesn't exist a library in Rust that will calculate this, so I'm on my own.



So one further problem are the weights. For the regression itself they were not required, but for the errors they are needed (see the R notebook). So actually, each point we have been using so far is actually 6 independant measurements and the malfuction shows how many malfunctioned.
To compute the correct standard erorrs, we crated 6 points out of every point. Of those, there will be `malfunction` points with `malfuction = 1`, and `6 - malfunction` points with `malfuction = 0`.
Now we have a realy classification problem and can solve this like one.


In [20]:

let result : Vec<Row> = result.iter().map(|result| {
    let mut v : Vec<Row> = Vec::new();
    for i in 0..result.malfunction {
        let mut x = result.clone();
        x.malfunction = 1;
        v.push( x);
    }
    for i in result.malfunction..result.count {
        let mut x = result.clone();
        x.malfunction = 0;
        v.push( x);
    }
    v
}).flatten().collect();

First, we want to get rid of this rather annoying statistics library. Whe already got the correct values for the regression, so let's build that ourself. This is also a nice example to show how the logistic regression curve is implemented:

In [28]:
// Our values from the logistic regression

let b_0: f64 = 5.084925304946187;
let b_1: f64= -0.11560036448787319;

fn logit(b_0: f64, b_1: f64, x: f64) -> f64 {
    let e: f64 = 2.718281828459045;
    let t = b_0 + b_1 * x;
    1.0 / (1.0 + e.powf(-t))
}


The next step will calculate the coveriance. Since my knowledge of the calculation required for logistic regression here is nonexistend, I used a python example found on the internet [0] and translated it into Rust

[0] https://stats.stackexchange.com/questions/89484/how-to-compute-the-standard-errors-of-a-logistic-regressions-coefficients

I will now try to do the same thing in Rust. I know it's not best practice to blindly take code from the internet, but the only alternative would be to read a statistics book, to which I don't really have time

In [29]:
let temperatures : Vec<f64> = result.iter().map(|x| x.temperature as f64).collect();
// First, we need the probabilities of each point belonging to our classes [No Malfunction, Malfunction]
let classProbs : Vec<(f64, f64)> = temperatures.iter().map(|x| logit(b_0, b_1, *x)).map(|p| (1.0 - p, p)).collect();
classProbs[0] // e.g. the first point has 92% change of not malfunctioning

(0.9272165637766812, 0.0727834362233189)

In [30]:
// Add [1] intercept to X_learn. Every temperatur now is a vector with [1, temperature]
let X_design : Vec<Vec<f64>> = result.iter().map(|x| vec!(1.0,x.temperature as f64)).collect();

In [31]:
// Take predictions, multiply both class values, put them in the diagonal of the matrix
// We use the matrix module from rusty_machine here
let m = Matrix::from_diag(
    &classProbs.iter().map(|(p, notP)| p * notP).collect::<Vec<f64>>()
);


In [32]:
// Looks like [[1, temp1], [1, temp2], ... [1, tempN]]. This is X_design from the Python code
let X_matrix: Matrix<f64> = Matrix::new(X_design.len(), 2,
    X_design.iter().flatten().map(|x| *x).collect::<Vec<f64>>());

// Transposed Matrix (the library can't transpose matrices that are not quadratic)
// Looks like [[1,1,1...,1], [temp1, temp2..., tempN]]
let X_matrix_T: Matrix<f64> = Matrix::new(2, X_design.len(),
    (0..result.len()).into_iter().map(|x| 1.0).chain(result.iter().map(|r| r.temperature as f64)).collect::<Vec<f64>>()
   );


In [33]:
// The final matrix multiplication. the .clone() functions are again because of lifetime problems
let covit = (X_matrix_T.clone() * m.clone() * X_matrix.clone()).inverse().unwrap();
covit

Matrix { rows: 2, cols: 2, data: [9.317664537301388, -0.14256542320020854, -0.1425654232002085, 0.002211238981360064] }

In [34]:
// Final step: The standard errors are square root of the diaganals of the covariance matrix:
&covit.diag().iter().map(|x| x.sqrt()).collect::<Vec<f64>>()

[3.0524849774079788, 0.04702381291813823]

Those are finally the correct standard errors! So we have managed to reproduce a bit more of the study now.

Even though I could continue trying to find the confidence interval (the relation between standard errors and confidence intervals seems not to be so simple in logistic regression), I chose to stop here due to a lack of time and motivation. More about that in the conclusion:

# Conclusion
Rust *can* theoretically used for exploring and analyzing data. This notebooks shows that. However, it's a lot harder than in languages like R or Python:
- The strong type system in Rust feels more like a hindrance than a help for exploring data. The type system that helps producing robust code is often a hindrance here and fighting against type-checking is very time-consuming. In the end, I feel that the Rust code is more robust than e.g. Python, but the cost in development is just too high.
- Libraries for data science lack a lot of features. The few existing ones are hard to use, lack examples require a lot of manual debugging to show what's going on. I choose to use rusty_machine, which looked to be almost the only useably canditade. In fact, the project has been abandoned since ca. 2017 :(.
- The Jupyter integration in Rust has some issues as well:
 - Since Rust is a compiled language, they have used a few hacks to make this work, e.g. a value-store for all variables to circumvent the lifetime and borrowing issues. Sometimes this reqires you to make extra .copy() calls, sometimes valid Rust code will just not compile at all. Imports also have their own problems.
 - When a process panic()s, you can recompile everything. Panic()s are usually rare, but especially rust_machines Matrix class calls panic() whenever dimensions don't match. And while experimenting, this happens a lot.
 - Recompiling means recompililing all dependecies as well. Especially 'plotters' has a lot of further dependencies, meaning everything takes a few minutes to recompile :(
 - Running cells is rather slow. This is because Jupyter will - each time code is run - transpile it into normal Rust, put that into a cargo project, compile and run it, and then copy over the output. Here, interpreted languages really have an advantage

All in all, I would strongly advise not to use Rust for interactive, exploratory data scince project. It has a lot of potential, but is still far from an acceptable experience
